# Lab 3: AgentCore Gateway를 사용하여 에이전트에 도구를 안전하게 연결하기

## 개요

이 랩에서는 Amazon Bedrock Gateway를 사용하여 조직에서 사용 가능한 도구를 고객 지원 에이전트와 통합하는 방법을 배웁니다.

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro)는 애플리케이션이 도구와 컨텍스트를 대규모 언어 모델(LLM)에 제공하는 방법을 표준화하는 개방형 프로토콜입니다.

[Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)를 사용하면 개발자는 API, Lambda 함수 및 기존 서비스를 MCP 호환 도구로 변환하고 몇 줄의 코드만으로 Gateway 엔드포인트를 통해 에이전트에게 제공할 수 있습니다.


**워크샵 여정:**

- **Lab 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **Lab 2 (완료):** 메모리 기능 강화 - 대화 맥락 및 개인화 추가
- **Lab 3 (현재):** 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구를 안전하게 공유
- **Lab 4:** 프로덕션 배포 - 관측 기능을 갖춘 AgentCore 런타임 사용
- **Lab 5:** 에이전트 성능 평가 - 온라인 평가로 품질 모니터링
- **Lab 6:** 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성


### AgentCore Gateway 및 도구 공유가 중요한 이유

현재 상태 (Lab 1-2): 각 에이전트에는 자체 도구 사본이 있습니다. 이는 확장 가능하지 않으며 다음과 같은 문제로 이어집니다.

- 서로 다른 에이전트 간의 코드 중복
- 일관성 없는 도구 동작 및 유지 관리 오버헤드
- 중앙화된 보안 또는 액세스 제어 없음
- 여러 사용 사례로 확장하기 어려움

이 랩을 마치면 다음을 제공할 수 있는 중앙화되고 재사용 가능한 도구를 갖게 됩니다.

- 고객 지원 에이전트 (현재 사용 사례)
- 영업 에이전트 (동일한 제품 정보 및 고객 데이터 필요)
- 재고 에이전트 (동일한 제품 정보 및 보증 확인 필요)
- 반품 처리 에이전트 (반품 정책 및 고객 프로필 필요)

및 기타 사용 사례. 

### AgentCore Identity로 안전한 인증 추가하기

또한 AgentCore Gateway는 인바운드 및 아웃바운드 연결 모두를 안전하게 인증해야 합니다. [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html)는 AWS 서비스 및 Slack 및 Zoom과 같은 타사 애플리케이션 전반에 걸쳐 원활한 에이전트 ID 및 액세스 관리를 제공하며 Okta, Entra 및 Amazon Cognito와 같은 표준 ID 공급자를 지원합니다. 이 랩에서는 AgentCore Gateway가 AgentCore Identity와 통합되어 인바운드 및 아웃바운드 인증을 통해 안전한 연결을 제공하는 방법을 확인합니다. 

인바운드 인증의 경우 AgentCore Gateway는 호출 중에 전달된 OAuth 토큰을 분석하여 게이트웨이의 도구에 대한 액세스를 허용하거나 거부합니다. 도구가 외부 리소스에 액세스해야 하는 경우 AgentCore Gateway는 API 키, IAM 또는 OAuth 토큰을 통한 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.

인바운드 권한 부여 흐름 중에 에이전트 또는 MCP 클라이언트는 사용자 IdP에서 생성된 OAuth 액세스 토큰을 추가하여 AgentCore Gateway의 MCP 도구를 호출합니다. 그런 다음 AgentCore Gateway는 OAuth 액세스 토큰을 검증하고 인바운드 권한 부여를 수행합니다.

AgentCore Gateway에서 실행되는 도구가 외부 리소스에 액세스해야 하는 경우 OAuth는 게이트웨이 타겟에 대한 리소스 자격 증명 공급자를 사용하여 다운스트림 리소스의 자격 증명을 검색합니다. AgentCore Gateway는 다운스트림 API에 액세스하기 위해 호출자에게 인증 자격 증명을 전달합니다.


## Lab 3 아키텍처

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*웹 검색 도구는 이제 안전한 ID 기반 액세스 제어를 통해 AgentCore Gateway에 중앙 집중화됩니다. 여러 에이전트 및 사용 사례가 동일한 도구를 안전하게 공유할 수 있습니다. 또한 다른 애플리케이션용으로 구축된 `check_warranty()` 도구를 재사용하고 다른 애플리케이션 내에서 사용할 `web_search()` 도구를 추가할 것입니다. `get_product_info()`, `get_return_policy()` 및 `get_technical_support`는 고객 지원 사용 사례에만 해당하므로 로컬 도구로 유지됩니다.*

### 주요 기능
- **AWS Lambda 함수 원활하게 통합:** 이 예제는 Amazon Bedrock AgentCore Gateway를 사용하여 기존 AWS Lambda 함수와 에이전트를 통합하여 항목의 보증을 확인하고 고객 프로필을 가져오는 방법을 보여줍니다.
- **인바운드 인증으로 Gateway 엔드포인트 보안:** 유효한 JWT 토큰을 제공하는 에이전트만 엔드포인트에 연결하여 도구를 사용할 수 있습니다.
- **MCP 엔드포인트를 사용하도록 에이전트 구성:** 에이전트는 유효한 JWT 토큰을 가져와 AgentCore Gateway에서 제공하는 MCP 엔드포인트에 연결하는 데 사용합니다.

## 사전 필수 조건

* Python 3.12+
* AWS 자격 증명 구성됨
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 Anthropic Claude 3.7 활성화됨
* Lab 2 완료: 고객 지원 에이전트에 메모리 추가
* 이 리소스는 AWS 워크샵 계정 내에서 생성됩니다.
    - AWS Lambda 함수 
    - AWS Lambda 실행 IAM 역할
    - AgentCore Gateway IAM 역할
    - AWS Lambda 함수에서 사용하는 DynamoDB 테이블. 
    - Cognito 사용자 풀 및 사용자 풀 클라이언트

## 단계 1: 필요한 라이브러리 가져오기

In [1]:
# Import libraries
import os
import sys
import boto3
import json

from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from lab_helpers.utils import (
    get_or_create_cognito_pool,
    put_ssm_parameter,
    get_ssm_parameter,
    load_api_spec,
)


sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']
# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 단계 2: 에이전트에게 기존 고객 데이터에 액세스할 수 있는 도구 제공하기
AgentCore Gateway는 세 가지 주요 방식으로 에이전트 도구 통합을 단순화합니다:

범용 MCP 지원: AgentCore Gateway의 MCP 표준을 통해 도구를 노출하여 모든 에이전트 프레임워크와 즉시 호환되도록 만듭니다.

간단한 REST 통합: 기존 REST 서비스를 AgentCore Gateway 타겟으로 추가하기만 하면 에이전트 도구로 변환할 수 있습니다.

Lambda 유연성: Lambda 함수를 API를 호출할 수 있는 MCP 엔드포인트로 노출합니다 - 여기서는 보증 상태를 확인하는 함수로 시연됩니다.

AgentCore Gateway는 호출할 도구의 이름으로 Lambda 컨텍스트를 채우고, 도구에 전달된 매개변수는 Lambda 이벤트에 제공됩니다:

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda 함수](./prerequisite/lambda/python/lambda_function.py)

```
def lambda_handler(event, context):
    if get_tool_name(event) == "check_warranty_status":
        serial_number = get_named_parameter(event=event, name="serial_number")
        customer_email = get_named_parameter(event=event, name="customer_email")

        warranty_status = check_warranty_status(serial_number, customer_email)
        return {"statusCode": 200, "body": warranty_status}
```

## 단계 3: 웹 검색 도구를 MCP로 변환하기
이제 AgentCore Gateway를 사용하여 MCP 서버를 개발하고 있으므로 여러 에이전트에서 사용할 것으로 생각되는 도구를 MCP화할 수 있습니다. 이러한 도구 중 하나는 Lab 1에서 구축한 웹 검색 도구일 수 있습니다. 결과적으로 Lab 1의 웹 검색 도구도 AgentCore Gateway 내의 Lambda 도구로 변환했습니다:

[웹 검색 Lambda](./prerequisite/lambda/python/web_search.py)
```
from ddgs import DDGS


def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc.
        max_results (int): The maximum number of results to return.
        
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except Exception as e:
        return f"Search error: {str(e)}"


print("✅ Web search tool ready")
```

## 단계 4: 함수 정의 메타데이터 생성하기
마지막으로, Lambda 함수에 의해 구현된 도구를 설명하는 도구 스키마를 작성해야 합니다.

이 파일은 이미 [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)에 정의되어 있습니다.

```
[
    {
        "name": "check_warranty_status",
        "description": "Check the warranty status of a product using its serial number and optionally verify via email",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {
                    "type": "string"
                },
                "customer_email": {
                    "type": "string"
                }
            },
            "required": [
                "serial_number"
            ]
        }
    },
    {
        "name": "web_search",
        "description": "Search the web for updated information using DuckDuckGo",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {
                    "type": "string",
                    "description": "The search query keywords"
                },
                "region": {
                    "type": "string",
                    "description": "The search region (e.g., us-en, uk-en, ru-ru)"
                },
                "max_results": {
                    "type": "integer",
                    "description": "The maximum number of results to return"
                }
            },
            "required": [
                "keywords"
            ]
        }
    }
]
```

## 단계 5. AgentCore Gateway 생성하기

이제 AgentCore Gateway를 생성하여 Lambda 함수를 MCP 호환 엔드포인트로 노출해 보겠습니다.

도구를 호출할 권한이 있는 호출자를 검증하려면 인바운드 인증을 구성해야 합니다.

인바운드 인증은 MCP 서버의 표준인 OAuth 권한 부여를 사용하여 작동합니다. OAuth를 사용하면 클라이언트 애플리케이션은 게이트웨이를 사용하기 전에 OAuth 권한 부여자로 인증해야 합니다. 클라이언트는 런타임에 사용되는 액세스 토큰을 받게 됩니다.

OAuth 검색 서버와 클라이언트 ID를 지정해야 합니다. 워크샵과 함께 제공된 Cloudformation은 이미 Cognito UserPool 및 UserPoolClient를 프로비저닝했으며 검색 URL과 클라이언트 ID를 전용 SSM 매개변수에 저장했습니다.

In [2]:
gateway_name = "customersupport-gw"

cognito_config = get_or_create_cognito_pool(refresh_token=True)
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

try:
    # create new gateway
    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_name", gateway_name)
    put_ssm_parameter(
        "/app/customersupport/agentcore/gateway_arn", create_response["gatewayArn"]
    )
    put_ssm_parameter(
        "/app/customersupport/agentcore/gateway_url", create_response["gatewayUrl"]
    )
    print(f"✅ Gateway created successfully with ID: {gateway_id}")

except Exception:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    print(f"Found existing gateway with ID: {existing_gateway_id}")

    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway["id"]

Error getting secret: An error occurred (ResourceNotFoundException) when calling the GetSecretValue operation: Secrets Manager can't find the specified secret.
No existing cognito config found. Creating a new one..
{'UserPoolId': 'us-west-2_KfQ5VN165', 'ClientName': 'MCPServerPoolClient', 'ClientId': '4s9vqjrnm2co5a35705sriffi0', 'ClientSecret': 'j0vtvdetvj8q3f8jpc74qu68vug2n7mo7j6r8efa5uetdisvs23', 'LastModifiedDate': datetime.datetime(2026, 2, 7, 9, 32, 6, 449000, tzinfo=tzlocal()), 'CreationDate': datetime.datetime(2026, 2, 7, 9, 32, 6, 449000, tzinfo=tzlocal()), 'RefreshTokenValidity': 30, 'TokenValidityUnits': {}, 'ExplicitAuthFlows': ['ALLOW_USER_PASSWORD_AUTH', 'ALLOW_USER_SRP_AUTH', 'ALLOW_REFRESH_TOKEN_AUTH'], 'AllowedOAuthFlowsUserPoolClient': False, 'EnableTokenRevocation': True, 'EnablePropagateAdditionalUserContextData': False, 'AuthSessionValidity': 3}
Pool id: us-west-2_KfQ5VN165
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_KfQ5VN165/.well-known/o

## 단계 6. Lambda 함수 타겟 추가하기
이제 [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)에서 이전에 정의된 함수 정의를 사용하여 Agent Gateway 내에 Lambda 타겟을 생성합니다. 이것은 게이트웨이가 호스팅할 도구를 정의합니다.

게이트웨이를 사용하면 여러 타겟을 게이트웨이에 연결할 수 있으며 언제든지 게이트웨이에 연결된 타겟/도구를 변경할 수 있습니다. 각 타겟은 자체 자격 증명 공급자를 가질 수 있지만, 게이트웨이는 단일 MCP URL이 되어 에이전트가 무수한 API 전반에 걸쳐 모든 관련 도구에 액세스할 수 있게 합니다.

In [3]:
try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API specification file not found: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)

    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter(
                    "/app/customersupport/agentcore/lambda_arn"
                ),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway target created: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ Error creating gateway target: {str(e)}")

✅ Gateway target created: DJNCO6OBUO


## 단계 7: 지원 에이전트에 새로운 MCP 기반 도구 추가하기
여기서는 Cognito의 인증 토큰을 Strands SDK의 MCPClient에 통합하여 Strands 에이전트와 통합할 MCP 서버 객체를 생성합니다.
### 단계 7.1. 안전한 MCP 클라이언트 객체 설정하기

In [4]:
print(f"Gateway Endpoint - MCP URL: {gateway['gateway_url']}")
# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway["gateway_url"],
        headers={"Authorization": f"Bearer {cognito_config['bearer_token']}"},
    )
)

with mcp_client:
    tools = mcp_client.list_tools_sync()
    print(f"   Found {len(tools)} tool(s):\n")
    for tool in tools:
        print(f"   ✅ {tool.mcp_tool.name}")
        print(f"      {tool.mcp_tool.description}\n")

Gateway Endpoint - MCP URL: https://customersupport-gw-wubxeu6p8c.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp
   Found 2 tool(s):

   ✅ LambdaUsingSDK___check_warranty_status
      Check the warranty status of a product using its serial number and optionally verify via email

   ✅ LambdaUsingSDK___web_search
      Search the web for updated information using DuckDuckGo



## 단계 7.2. MCP 클라이언트를 사용하여 에이전트의 도구에 액세스하기
이제 이전 랩의 리소스와 함께 구축한 AgentCore Gateway를 사용하여 Strands 에이전트를 생성합니다. 이제 에이전트는 Strands 에이전트를 통한 로컬 도구와 AgentCore Gateway를 통한 MCP 도구를 혼합하여 사용합니다.

In [ ]:
from lab_helpers.lab1_strands_agent import (
    get_product_info,
    get_return_policy,
    get_technical_support,
    SYSTEM_PROMPT,
)

import uuid
from lab_helpers.lab2_memory import create_or_get_memory_resource
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig, RetrievalConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

memory_id = create_or_get_memory_resource()

SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_001"

memory_config = AgentCoreMemoryConfig(
        memory_id=memory_id,
        session_id=str(SESSION_ID),
        actor_id=CUSTOMER_ID,
        retrieval_config={
            "support/customer/{actorId}/semantic/": RetrievalConfig(top_k=3, relevance_score=0.2),
            "support/customer/{actorId}/preferences/": RetrievalConfig(top_k=3, relevance_score=0.2)
        }
    )

# Initialize the Bedrock model
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION,
)


def create_agent(prompt):
    try:
        with mcp_client:
            tools = [
                get_product_info,
                get_return_policy,
                get_technical_support,
            ] + mcp_client.list_tools_sync()

            # Create the customer support agent
            agent = Agent(
                model=model,
                tools=tools,
                system_prompt=SYSTEM_PROMPT,
                session_manager=AgentCoreMemorySessionManager(memory_config, REGION),
            )
            response = agent(prompt)
            return response
    except Exception as e:
        raise e


print("✅ Customer support agent created successfully!")

## 단계 8: 기존 API에 대한 MCP 도구 액세스로 에이전트 테스트하기

모든 기능이 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해 봅시다. 출력에 5개의 도구가 표시되어야 합니다.

In [ ]:
test_prompts = [
    # Warranty Checks
    "List all of your tools",
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?",
    "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333.",
    "What are the warranty support guidelines?",
    "How can I fix Lenovo Thinkpad with a blue screen",
    "Tell me detailed information about the technical documentation on installing a new CPU",
]


# Function to test the agent
def test_agent_responses(prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nTest Case {i}: {prompt}")
        print("-" * 50)
        try:
            response = create_agent(prompt)
            print(response)
        except Exception as e:
            print(f"Error: {str(e)}")
        print("-" * 50)


# Run the tests
test_agent_responses(test_prompts)

print("\\n✅ Basic testing completed!")

## [선택 섹션] AgentCore Policy

### 단계 9: 정책 엔진 생성하기

세분화된 액세스 제어를 위한 Cedar 권한 부여 정책을 포함할 정책 엔진을 생성합니다.

In [ ]:
# Try to import from toolkit, fall back to custom implementation if not available
try:
    from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient
    print("✅ Using toolkit PolicyClient")
except ImportError:
    from utils.policy_utils import PolicyClient
    print("✅ Using custom PolicyClient (toolkit policy module not available)")

# Initialize the policy client
policy_client = PolicyClient(region_name=REGION)

print("\n🔧 Creating Policy Engine...")

# Create or get existing policy engine
# The policy engine is a container for all our authorization policies
engine = policy_client.create_or_get_policy_engine(
    name="customersupport_pe",
    description="Policy engine for customer support gateway",
)

engine_id = engine['policyEngineId']
engine_arn = engine['policyEngineArn']
put_ssm_parameter("/app/customersupport/agentcore/policy_engine_id", engine_id)

print(f"\n✅ Policy Engine ready")
print(f"   Engine ID: {engine_id}")
print(f"   Engine ARN: {engine_arn}")

### [선택 사항]: 자연어를 사용하여 Cedar 정책 생성하기

AgentCore Policy는 자연어를 사용하여 범위 기반 액세스를 시행하기 위한 Cedar 정책을 생성하는 기능을 제공합니다.

In [ ]:
import time
# Create Cedar policy for approve tool (write scope)
print("\n📝 Generating Cedar Policy from Natural language...")

nl_input = "Allow tag username == 'testuser' to perform check warranty status on the customer support gateway."

warranty_tool_policy = policy_client.generate_policy(
    policy_engine_id=engine["policyEngineId"],
    name=f"nl_policy_{int(time.time())}",
    resource={"arn": gateway["gateway_arn"]},
    content={"rawText": nl_input},
    fetch_assets=True,
)

print("✅ Policy generated from natural language")

In [ ]:
print("📋 Generated Cedar Policies:\n")
print("=" * 80)

# Allow warranty status policy
print("\n1️⃣  Warranty Status")
print("-" * 80)
warranty_tool_policy_cedar = warranty_tool_policy["generatedPolicies"][0]["definition"]["cedar"]["statement"]
print(warranty_tool_policy_cedar)

print("\n" + "=" * 80)

### 단계 10 : Cedar 정책 생성하기

범위 기반 액세스를 시행하기 위한 Cedar 정책을 생성해 봅시다. 승인 작업을 위한 쓰기 범위, 생성/목록 작업을 위한 읽기 범위를 작성할 수 있습니다.

이 워크샵의 범위에서는 모든 사용자의 일관성을 보장하기 위해 미리 만들어진 허용/거부 cedar 정책을 사용합니다.

In [ ]:
# Providing policies to allow for consistency in use case execution
allow_policy = {
    "cedar": {
        "statement": f"""permit(
            principal,
            action in [AgentCore::Action::"LambdaUsingSDK___check_warranty_status", AgentCore::Action::"LambdaUsingSDK___web_search"],
            resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:{REGION}:{account_id}:gateway/{gateway['id']}"
        ) when {{
            (principal.hasTag("username")) && 
            ((principal.getTag("username")) == "testuser")
        }};"""
    }
}

# deny web search for "iPhone 8" keywords
deny_web_search_policy = {
    "cedar": {
        "statement": f"""forbid(
            principal,
            action == AgentCore::Action::"LambdaUsingSDK___web_search",
            resource == AgentCore::Gateway::"arn:aws:bedrock-agentcore:{REGION}:{account_id}:gateway/{gateway['id']}"
        ) when {{
            context.input has keywords &&
            context.input.keywords like "*iPhone 8*"
        }};"""
    }
}

### 단계 11: 정책 엔진에 정책 추가하기

생성된 Cedar 정책을 정책 엔진에 생성합니다.

In [ ]:
print("🔧 Creating policies in Policy Engine...\n")

# Create allow both tools policy
warranty_result = policy_client.create_or_get_policy(
    policy_engine_id=engine["policyEngineId"],
    name="allow_policy",
    description="Allow web_search and check_warranty_status calls",
    definition=allow_policy
)
print("✅ Policy ready: allow_policy")
print("   Tools allowed: check_warranty_status and web_search\n")

# Create deny web search list policy
web_search_deny_result = policy_client.create_or_get_policy(
    policy_engine_id=engine["policyEngineId"],
    name="deny_web_search",
    description="Deny web_search tool call for iPhone 8",
    definition=deny_web_search_policy
)
print("✅ Policy ready: deny_web_search")
print("   Tools denied conditionally: web_search\n")

print("✅ All policies ready!")

### 단계 12: 게이트웨이 IAM 역할 권한 업데이트하기

정책 엔진을 연결하기 전에 게이트웨이 IAM 역할에 정책 엔진에 액세스할 수 있는 권한을 부여해야 합니다.

In [ ]:
role_arn = get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role")
role_name = role_arn.split('/')[-1] 

iam_client = boto3.client('iam')
print("🔧 Updating Gateway IAM role with Policy Engine permissions...")

# Policy document that grants access to Policy Engine
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:*"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{REGION}:{account_id}:policy-engine/*",
                f"arn:aws:bedrock-agentcore:{REGION}:{account_id}:gateway/*"
            ]
        }
    ]
}

try:
    # Add inline policy to the role
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="PolicyEngineAccess",
        PolicyDocument=json.dumps(policy_document)
    )
    
    print(f"✅ IAM role updated successfully")
    print(f"   Role: {role_name}")
    print(f"   Added permissions: GetPolicyEngine, GetPolicy, ListPolicies")
    print("\n⏳ Waiting 10 seconds for IAM changes to propagate...")
    time.sleep(10)
    
    print("✅ Ready to attach Policy Engine")
    
except Exception as e:
    print(f"❌ Error updating IAM role: {e}")
    print("\nYou may need to manually add these permissions to the role.")

### 단계 13: 게이트웨이에 정책 엔진 연결하기

정책 엔진을 ENFORCE 모드로 게이트웨이에 연결하여 정책 시행을 활성화합니다.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

# Initialize gateway client
gateway_client_toolkit = GatewayClient(region_name=REGION)

print("🔧 Attaching Policy Engine to Gateway...")
print(f"   Mode: ENFORCE (policies will block unauthorized requests)\n")

# Attach the policy engine to the gateway
update_response = gateway_client_toolkit.update_gateway_policy_engine(
    gateway_identifier=gateway["id"],
    policy_engine_arn=engine["policyEngineArn"],
    mode="ENFORCE"
)

print("✅ Policy Engine attached successfully!")
print(f"   Gateway ID: {gateway['id']}")
print(f"   Policy Engine: {engine['policyEngineId']}")
print(f"   Mode: ENFORCE")
print("\n🔒 Authorization is now active!")

### 단계 14: 정책 시행 테스트하기

In [ ]:
test_prompts = [
    "List all of your tools",
    "Search the web for heating issues with Samsung zfold 7",
    "Search the internet for heating issues with iPhone 8"
]

# Function to test the agent
def test_agent_responses(prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nTest Case {i}: {prompt}")
        print("-" * 50)
        try:
            response = create_agent(prompt)
            print(response)
        except Exception as e:
            print(f"Error: {str(e)}")
        print("-" * 50)

# Run the tests
test_agent_responses(test_prompts)

print("\\n✅ Policy testing completed!")

### 축하합니다! 🎉


Lab 3: AgentCore Gateway를 사용하여 도구를 에이전트에 안전하게 연결하기를 성공적으로 완료했습니다

달성한 내용:

##### 도구 중앙화 및 재사용성:

- 웹 검색을 로컬 도구에서 중앙화된 AgentCore Gateway로 마이그레이션했습니다
- 기존 엔터프라이즈 Lambda 함수 통합 (보증 확인, 고객 프로필)
- 여러 에이전트 유형이 액세스할 수 있는 공유 도구 인프라 생성

##### 엔터프라이즈급 보안:

- Cognito 통합으로 JWT 기반 인증 구현
- 게이트웨이 액세스를 위한 안전한 인바운드 권한 부여 구성
- 도구 사용을 위한 아이덴티티 기반 액세스 제어 수립

##### 확장 가능한 아키텍처 기반:

- 여러 사용 사례(고객 지원, 영업, 반품 처리)를 지원하는 재사용 가능한 도구 구축
- 서로 다른 에이전트 간의 코드 중복 제거
- 도구 업데이트 및 유지 관리를 위한 중앙화된 관리 생성

##### 현재의 한계 (다음 랩에서 해결할 예정!):

- **로컬 개발 환경** - 여전히 노트북에서 실행 중이며, 프로덕션 준비가 되지 않음
- **제한된 관측성** - 에이전트 동작 및 성능에 대한 포괄적인 모니터링 부재
- **수동 확장** - 증가된 부하 또는 여러 동시 사용자를 자동으로 처리할 수 없음

##### 다음 단계: Lab 4 - AgentCore 런타임으로 프로덕션에 배포하기

Lab 4에서는 프로토타입을 다음과 같은 프로덕션 준비 시스템으로 변환합니다:

- 확장 가능한 에이전트 배포를 위한 AgentCore 런타임
- 메트릭, 로깅 및 추적을 포함한 포괄적인 관측성
- 실제 트래픽을 처리하기 위한 자동 확장 기능

### 리소스
- [Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)
- [Strands Agents 설명서](https://github.com/strands-agents/sdk-python)
- [공식 고객 지원 샘플](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)